In [75]:
import random
import math
import cmath

In [76]:
# Define the Node Structure
class Node:
    def __init__(self, type, value=None):
        self.type = type
        self.value = value
        self.children = []

    def evaluate(self, input_values):
        if self.value == '+':
            return self.children[0].evaluate(input_values) + self.children[1].evaluate(input_values)
        elif self.value == '-':
            return self.children[0].evaluate(input_values) - self.children[1].evaluate(input_values)
        elif self.value == '*':
            return self.children[0].evaluate(input_values) * self.children[1].evaluate(input_values)
        elif self.value == '/':
            denominator = self.children[1].evaluate(input_values)
            if denominator == 0:
                return 0  # Handle division by zero
            return self.children[0].evaluate(input_values) / denominator
        elif self.value == '^':
            base = self.children[0].evaluate(input_values)
            exponent = self.children[1].evaluate(input_values)
            if isinstance(base, complex) or isinstance(exponent, complex):
                return complex(base) ** complex(exponent)
            elif base == 0 and (exponent < 0 or exponent.imag != 0):
                return 0  # Return a default value for zero base raised to negative or complex exponent
            try:
                return base ** exponent
            except OverflowError:
                return 0  # Handle overflow error
        elif self.value == 'sin':
            argument = self.children[0].evaluate(input_values)
            try:
                return math.sin(argument) if isinstance(argument, float) else cmath.sin(argument)
            except OverflowError:
                return 0  # Handle overflow error
        elif self.value == 'cos':
            argument = self.children[0].evaluate(input_values)
            try:
                return math.cos(argument) if isinstance(argument, float) else cmath.cos(argument)
            except OverflowError:
                return 0  # Handle overflow error
        elif self.value in input_values:
            return input_values[self.value]
        else:
            return float(self.value)

In [77]:
# Generate Initial Population
def generate_tree(max_depth, available_operators, available_variables):
    if max_depth == 0 or random.random() < 0.1:
        if random.random() < 0.5:
            return Node('constant', value=random.uniform(-10, 10))
        else:
            return Node('variable', value=random.choice(available_variables))
    else:
        node = Node('operator', value=random.choice(available_operators))
        num_children = 2 if node.value not in ['sin', 'cos'] else 1
        for _ in range(num_children):
            node.children.append(generate_tree(max_depth - 1, available_operators, available_variables))
        return node

In [78]:
# Define the Fitness Function
def fitness_function(tree, input_data, target_outputs):
    total_error = 0.0
    for input_values, target_output in zip(input_data, target_outputs):
        output = tree.evaluate(input_values)
        error = abs(output - target_output)
        total_error += error
    return total_error

In [79]:
# Genetic Operators
def crossover(parent1, parent2):
    child = parent1
    if parent1.children and parent2.children:
        node1 = random.choice(parent1.children)
        node2 = random.choice(parent2.children)
        node1_index = parent1.children.index(node1)
        node2_index = parent2.children.index(node2)
        child.children[node1_index] = node2
        child.children[node2_index] = node1
    return child

In [80]:
def mutation(tree, max_depth, available_operators, available_variables):
    mutated_tree = tree
    node = random.choice(mutated_tree.children)
    node_index = mutated_tree.children.index(node)
    mutated_tree.children[node_index] = generate_tree(max_depth, available_operators, available_variables)
    return mutated_tree

In [81]:
# Evolutionary Loop
def genetic_programming(input_data, target_outputs, available_operators, available_variables, population_size, max_depth, num_generations):
    population = []
    for _ in range(population_size):
        tree = generate_tree(max_depth, available_operators, available_variables)
        population.append(tree)

    for generation in range(num_generations):
        # Evaluate fitness for each individual
        fitness_scores = []
        for individual in population:
            fitness_scores.append(fitness_function(individual, input_data, target_outputs))

        # Select parents for reproduction
        parents = random.choices(population, weights=[1 / (score + 1) for score in fitness_scores], k=2)

        # Create offspring through crossover and mutation
        offspring = []
        for _ in range(population_size - 1):
            child = crossover(parents[0], parents[1])
            child = mutation(child, max_depth, available_operators, available_variables)
            offspring.append(child)

        # Replace the least fit individual with the best individual to maintain diversity
        best_individual_index = fitness_scores.index(min(fitness_scores))
        offspring.append(population[best_individual_index])

        population = offspring

    # Get the best individual from the final population
    fitness_scores = [fitness_function(individual, input_data, target_outputs) for individual in population]
    best_individual_index = fitness_scores.index(min(fitness_scores))
    best_individual = population[best_individual_index]

    return best_individual

In [82]:
# Example Usage
input_data = [[1], [2], [3], [4], [5]]
target_outputs = [3, 5, 7, 9, 11]
available_operators = ['+', '-', '*', '/', '^', 'sin', 'cos']
available_variables = [0]  # Assuming only one variable 'x'
population_size = 100
max_depth = 3
num_generations = 50

best_individual = genetic_programming(input_data, target_outputs, available_operators, available_variables, population_size, max_depth, num_generations)

# Print the best individual and its fitness score
print("Best Individual:", best_individual)
fitness_score = fitness_function(best_individual, input_data, target_outputs)
print("Fitness Score:", fitness_score)

Best Individual: <__main__.Node object at 0x7a9352aae9e0>
Fitness Score: 34.82781847388725


In [83]:
# # Generate training data
# np.random.seed(1)
# X = np.random.uniform(-2*np.pi, 2*np.pi, 1000)  # Input features with 1000 samples
# y = np.sin(X)  # Target variable is the sine of X